In [1]:
import base64
import requests as rq
import json

In [2]:
def get_oauth_token():
    '''
    This function will return our personalised token
    '''
    api_key = 'sj8re42xlfdmnucq62vr6o80s3tpzlgr'   # Your API key provided by Idealista
    secret = 'JNZRAny0BDYd'   # Your secred code provided by Idealista

    message = api_key + ":" + secret   # Combine the API key and the secret to get our personalised message

    auth = "Basic " + base64.b64encode(message.encode("ascii")).decode("ascii")   # Encode the message

    headers_dic = {"Authorization" : auth,
                   "Content-Type" : "application/x-www-form-urlencoded;charset=UTF-8"}   # Define our headers

    params_dic = {"grant_type" : "client_credentials",   # Define the request params
                  "scope" : "read"}

    r = rq.post("https://api.idealista.com/oauth/token",   # Perform the request with the api url, headers and params
                      headers = headers_dic,
                      params = params_dic)

    token = json.loads(r.text)['access_token']   # Obtain the personalised token, as a json

    return token

In [10]:
base_url = 'https://api.idealista.com/3.5/'     # Base search url
country = 'es'     # Search country (es, it, pt)                          
language = 'es'     # Search language (es, it, pt, en, ca) 
max_items = '50'     # Max items per call, the maximum set by Idealista is 50
operation = 'sale'     # Kind of operation (sale, rent) 
property_type = 'homes'     # Type of property (homes, offices, premises, garages, bedrooms)
order = 'priceDown'     # Order of the listings, consult documentation for all the available orders 
center = '36.52178262315535,-6.283122287029964'     # Coordinates of the search center
distance = '4000'     # Max distance from the center
sort = 'desc'     # How to sort the found items
bankOffer = 'false'     # If the owner is a bank
     # Max price of the listings

In [12]:
# Define a function to obtain our search url

def define_search_url():
    '''
    This function will combine our params with the url, in order to create our own search url
    '''
    url = (base_url +      
           country +
           '/search?operation=' + operation +
           '&maxItems=' + max_items +
           '&order=' + order +
           '&center=' + center +
           '&distance=' + distance +
           '&propertyType=' + property_type +
           '&sort=' + sort + 
           '&numPage=%s' +
           '&language=' + language)
    
    return url

url = define_search_url()

In [13]:
def search_api(url):  
    '''
    This function will use the token and url created previously, and return our search results.
    '''
    token = get_oauth_token()   #  Get the personalised token

    headers = {'Content-Type': 'Content-Type: multipart/form-data;',   # Define the search headers 
               'Authorization' : 'Bearer ' + token}

    content = rq.post(url, headers = headers)   # Return the content from the request

    result = json.loads(content.text)   # Transform the result as a json file

    return result

In [14]:
# Since we need to give pagination to our search and this is our first search, we will set the pagination as 1
pagination = 1
first_search_url = url %(pagination)

In [15]:
# Proceed to do the search with the paginated url
results = search_api(first_search_url)
# First of all, we can extract 50 results/page, but there are more pages, so we have to define how many pages there are.

total_pages = results['totalPages']

In [16]:
results añadir

{'elementList': [{'propertyCode': '102919557',
   'thumbnail': 'https://img3.idealista.com/blur/WEB_LISTING/0/id.pro.es.image.master/f6/cf/5a/1171784299.jpg',
   'externalReference': 'His-05734',
   'numPhotos': 33,
   'floor': '2',
   'price': 374000.0,
   'propertyType': 'flat',
   'operation': 'sale',
   'size': 270.0,
   'exterior': True,
   'rooms': 9,
   'bathrooms': 7,
   'address': 'Centro Histórico - Plaza España',
   'province': 'Cádiz',
   'municipality': 'Cádiz',
   'district': 'Centro Histórico - Plaza España',
   'country': 'es',
   'latitude': 36.5303433,
   'longitude': -6.2935765,
   'showAddress': False,
   'url': 'https://www.idealista.com/inmueble/102919557/',
   'distance': '1333',
   'description': 'Bienvenidos a este impresionante piso, licenciado como vivienda para fines turísticos, que ofrece una experiencia de alojamiento sin igual. Con una distribución única, este lugar ofrece comodidades y amplias habitaciones para que disfrutes de una estancia verdaderament

In [8]:
results

{'message': 'Invalid coordinates format. Latitude and longitude should be specified in decimal format and separated by a comma',
 'httpStatus': 400}

In [18]:
# Import the necessary libraries

import pandas as pd

In [19]:
def results_to_df(results):
    '''
    This function will save the json results as a dataframe and return the resulting dataframe
    '''
    df = pd.DataFrame.from_dict(results['elementList'])

    return df

In [20]:
def concat_df(df, df_tot):
    '''
    This function will take the main dataframe (df_tot), and concat it with the given individual dataframe, 
    returning the main dataframe 
    '''
    pd.concat([df_tot,df])
    
    return df_tot

In [21]:
# Proceed to save the obtained results as a dataframe
df = results_to_df(results)

In [22]:
results

{'elementList': [{'propertyCode': '102919557',
   'thumbnail': 'https://img3.idealista.com/blur/WEB_LISTING/0/id.pro.es.image.master/f6/cf/5a/1171784299.jpg',
   'externalReference': 'His-05734',
   'numPhotos': 33,
   'floor': '2',
   'price': 374000.0,
   'propertyType': 'flat',
   'operation': 'sale',
   'size': 270.0,
   'exterior': True,
   'rooms': 9,
   'bathrooms': 7,
   'address': 'Centro Histórico - Plaza España',
   'province': 'Cádiz',
   'municipality': 'Cádiz',
   'district': 'Centro Histórico - Plaza España',
   'country': 'es',
   'latitude': 36.5303433,
   'longitude': -6.2935765,
   'showAddress': False,
   'url': 'https://www.idealista.com/inmueble/102919557/',
   'distance': '1333',
   'description': 'Bienvenidos a este impresionante piso, licenciado como vivienda para fines turísticos, que ofrece una experiencia de alojamiento sin igual. Con una distribución única, este lugar ofrece comodidades y amplias habitaciones para que disfrutes de una estancia verdaderament

In [ ]:
# Since we still don't have a main dataframe where we can store all the data, we will create an empty dataframe
df_tot = pd.DataFrame()
df_tot = concat_df(df, df_tot)

In [ ]:
# This will loop over all the pages in the search results. It start from 2, because we've already gotten the first page
for i in range(2, total_pages): 

    url = url %(i)   # Add the pagination to the url
    results = search_api(url)   # Get the search results
    df = results_to_df(results)   # Save the results as a dataframe
    df_tot = concat_df(df, df_tot)   # Concat the results to the main dataframe

In [ ]:
# Once we have all our data, we just need to save it as a csv file, we have created the following function for that:

file_path = 'idealista.csv'

def df_to_csv(df):
    '''
    This function will take a given dataframe and save it as a csv file
    '''
    df = df.reset_index()   # Reset the index in order to organise the records
    df.to_csv(file_path, index=False)   # Save it into a csv

In [ ]:
# Run the function and you'll obtain a csv file with all the extracted data
df_to_csv(df_tot)